In [1]:
import torch

In [2]:

'''
For the fine-tuning of the VGG-Face network for the emotion
recognition task, we investigated various options in our preliminary
analysis. We found that combining weight decay and dropout for regularization
gives the best results on the FER validation set. We carry
out a multi-stage fine-tuning. In the first stage, we fine-tune on the
FER public test set, and run weight updates for five epochs. In the second
stage, we update the upper layers (higher than layer 27) using'''

''' We then fine-tune the VGG-face model on FER 2013
dataset, using both the training and the public test set; during
training we use data augmentation by jittering the scale, flipping
and rotating the faces. The aim is to make the network more robust
to small misalignment of the faces. We also apply a strong dropout
on the last layer of the VGG (keeping only 5% of the nodes) to
prevent over-fitting. We achieve a performance of 71.2% on the
FER private test set, which is slightly higher than the previously
published results '''

'''During the training of the deep networks, we oversample the training
images by rotating them around their center by a random angle between 
−15° and 15°, and by circularly shifting the images in the horizontal and 
vertical directions by an amount no more than 20% of the image size. 
This approach helps our network to be more robust against alignment errors. 
In Fig. 2, we show the training curves of two stages of fine-tuning of the 
network with the FER dataset, where we set the learning rate and weight 
decay to 0.0005, momentum to 0.9, and dropout probability to 0.8 [25].'''


'During the training of the deep networks, we oversample the training\nimages by rotating them around their center by a random angle between \n\xe2\x88\x9215\xc2\xb0 and 15\xc2\xb0, and by circularly shifting the images in the horizontal and \nvertical directions by an amount no more than 20% of the image size. \nThis approach helps our network to be more robust against alignment errors. \nIn Fig. 2, we show the training curves of two stages of fine-tuning of the \nnetwork with the FER dataset, where we set the learning rate and weight \ndecay to 0.0005, momentum to 0.9, and dropout probability to 0.8 [25].'

### VGG16-Face model

In [3]:
import VGG_FACE
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms

In [4]:
model = VGG_FACE.VGG_FACE

model.load_state_dict(torch.load('VGG_FACE.pth'))

#how to test whether this is pretrained?
#model.eval()


### Dataset: FERplus

In [5]:
import os
from skimage import io
import pandas as pd
import numpy as np
import argparse
import torch
from torch.utils.data import Dataset, DataLoader

In [6]:
df = pd.read_csv('all-data/FERPlus/fer2013new.csv')
df.dropna(axis=0, how='any')

,Usage,Image name,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
0,Training,fer0000000.png,4,0,0,1,3,2,0,0,0,0
1,Training,fer0000001.png,6,0,1,1,0,0,0,0,2,0
2,Training,fer0000002.png,5,0,0,3,1,0,0,0,1,0
3,Training,fer0000003.png,4,0,0,4,1,0,0,0,1,0
4,Training,fer0000004.png,9,0,0,1,0,0,0,0,0,0
5,Training,fer0000005.png,6,0,0,1,0,0,1,1,1,0
6,Training,fer0000006.png,2,0,0,8,0,0,0,0,0,0
7,Training,fer0000007.png,0,10,0,0,0,0,0,0,0,0
8,Training,fer0000008.png,0,10,0,0,0,0,0,0,0,0
9,Training,fer0000009.png,0,0,6,0,0,0,4,0,0,0


In [24]:
class FaceEmotionsDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform = None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.emotions_frame = pd.read_csv(csv_file)
        self.emotions_frame.dropna(axis=0, how='any', inplace=True)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.emotions_frame)

    def __getitem__(self, idx):
        img_name = self.emotions_frame.iloc[idx][1]
        
        img_path = os.path.join(self.root_dir, img_name)
        image = io.imread(img_path)
        #this takes the most highest ranked emotion. if two emotions have the same ranking, it just takes the first one
        emotion = np.argmax(self.emotions_frame.iloc[idx,2:].as_matrix())
        sample = {'image': image, 'emotion': emotion}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    
    

In [25]:
emotions_frame = pd.read_csv('all-data/FERPlus/fer2013new.csv')
emotions_frame.dropna(axis=0, how='any', inplace=True)

emotion = np.argmax(emotions_frame.iloc[10,2:].as_matrix())
print(emotion)

4


In [26]:
face_emotions = FaceEmotionsDataset(csv_file = 'all-data/FERPlus/fer2013new_training.csv', 
                                    root_dir = 'all-data/FERPlus/data/FER2013Train')

face_emotions[10]

{'emotion': 4,
 'image': array([[ 0.11764706,  0.09411765,  0.08235294, ...,  0.14509805,
          0.17254902,  0.14509805],
        [ 0.12156863,  0.08627451,  0.08235294, ...,  0.14509805,
          0.13725491,  0.16078432],
        [ 0.10588235,  0.08627451,  0.07450981, ...,  0.12941177,
          0.13333334,  0.15686275],
        ..., 
        [ 0.11372549,  0.11372549,  0.10196079, ...,  0.4627451 ,
          0.51764709,  0.58039218],
        [ 0.11764706,  0.11764706,  0.10588235, ...,  0.60392159,
          0.62352943,  0.65098041],
        [ 0.1254902 ,  0.11372549,  0.10980392, ...,  0.67450982,
          0.67843139,  0.67843139]], dtype=float32)}

In [10]:
import matplotlib.pyplot as plt
#for i in range(len(face_emotions)):
    #sample = face_emotions[i]

    #print(i, sample['image'].shape, sample['emotion'])
"""
    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_landmarks(**sample)

    if i == 3:
        plt.show()
        break
"""
#plt.imshow(face_emotions[10])
#plt.show()
#print(face_emotions[10])


"\n    ax = plt.subplot(1, 4, i + 1)\n    plt.tight_layout()\n    ax.set_title('Sample #{}'.format(i))\n    ax.axis('off')\n    show_landmarks(**sample)\n\n    if i == 3:\n        plt.show()\n        break\n"

In [13]:
from torchvision.datasets import ImageFolder
from skimage import transform
from PIL import Image
from skimage import io; io.use_plugin('matplotlib')

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, emotion = sample['image'], sample['emotion']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        #print(image.shape)
        #image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'emotion': emotion}

class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, emotion = sample['image'], sample['emotion']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        #emotion = emotion - [left, top]

        return {'image': image, 'emotion': emotion}

class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or tuple): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, emotion = sample['image'], sample['emotion']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        
        #emotion = emotion * [new_w / w, new_h / h]

        return {'image': img, 'emotion': emotion}


imgTransform = transforms.Compose([Rescale(256), #scale to 256x256
                                   #transforms.CenterCrop(224), #crops the image at center to 224x224
                                   RandomCrop(224),
                                   ToTensor()
                                   ])
                                   #, #turn the jpg/pil/wahtever image into a tensor
                                   #transforms.Normalize(mean = [0.485, 0.456, 0.406], #normalize with these vals
                                                        #std=[0.229, 0.224, 0.225])])
                                    ##HOW TO GET NORMALIZED VALUES?
                                    #to add: jitter/rotate data augmentation, flipping, 
                                   
#this doesn't work because the data is organized w a csv file w prob distrib of labels 
#instead of a single ground truth
#see this paper: https://arxiv.org/pdf/1608.01041.pdf
trainset = FaceEmotionsDataset(csv_file = 'all-data/FERPlus/fer2013new_training.csv', 
                                    root_dir = 'all-data/FERPlus/data/FER2013Train', transform = imgTransform)
valset = FaceEmotionsDataset(csv_file = 'all-data/FERPlus/fer2013new_validation.csv', 
                                    root_dir = 'all-data/FERPlus/data/FER2013Valid', transform = imgTransform)

trainLoader = torch.utils.data.DataLoader(trainset, batch_size = 64, 
                                          shuffle = True, num_workers = 0)
valLoader = torch.utils.data.DataLoader(valset, batch_size = 64, 
                                       shuffle = False, num_workers = 0)


In [35]:
for i in range(len(trainset)):
    sample = trainset[i]

    print(i, sample['image'], sample['emotion'])

    if i == 3:
        break

(0, 
 0.2070  0.2157  0.2225  ...   0.1872  0.1920  0.1968
 0.2214  0.2307  0.2382  ...   0.1788  0.1854  0.1919
 0.2351  0.2452  0.2537  ...   0.1713  0.1793  0.1872
          ...             ⋱             ...          
 0.2357  0.2215  0.2024  ...   0.3698  0.3466  0.3235
 0.2575  0.2458  0.2297  ...   0.3801  0.3622  0.3443
 0.2793  0.2701  0.2571  ...   0.3904  0.3778  0.3652
[torch.DoubleTensor of size 224x224]
, 0)
(1, 
 0.4837  0.4816  0.4795  ...   0.5300  0.5215  0.5131
 0.4713  0.4742  0.4770  ...   0.5362  0.5300  0.5237
 0.4589  0.4667  0.4745  ...   0.5425  0.5384  0.5344
          ...             ⋱             ...          
 0.1584  0.1286  0.0988  ...   0.7254  0.7254  0.7253
 0.2422  0.2058  0.1694  ...   0.7252  0.7253  0.7254
 0.3367  0.2937  0.2507  ...   0.7248  0.7250  0.7253
[torch.DoubleTensor of size 224x224]
, 0)
(2, 
 6.3086e-01  6.3976e-01  6.4865e-01  ...   1.0195e-01  9.5749e-02  9.3509e-02
 6.3544e-01  6.4198e-01  6.4853e-01  ...   1.0614e-01  9.9104e-02  

In [61]:
for (i, sample) in enumerate(trainLoader):
    inputs = sample['image']
    labels = sample['emotion']
    print("{}, {}".format(inputs, labels))



( 0 ,.,.) = 
  0.1164  0.1532  0.1900  ...   0.1389  0.1324  0.1259
  0.1164  0.1532  0.1900  ...   0.1432  0.1363  0.1294
  0.1164  0.1532  0.1900  ...   0.1476  0.1402  0.1329
           ...             ⋱             ...          
  0.0662  0.0871  0.1080  ...   0.0380  0.0373  0.0365
  0.0671  0.0883  0.1095  ...   0.0387  0.0380  0.0373
  0.0677  0.0891  0.1105  ...   0.0402  0.0395  0.0389

( 1 ,.,.) = 
  0.3563  0.4688  0.5814  ...   0.5998  0.5983  0.5968
  0.3568  0.4694  0.5821  ...   0.5998  0.5983  0.5968
  0.3572  0.4700  0.5828  ...   0.5998  0.5983  0.5968
           ...             ⋱             ...          
  0.3656  0.4810  0.5964  ...   0.4511  0.4452  0.4393
  0.3656  0.4810  0.5964  ...   0.4559  0.4479  0.4400
  0.3656  0.4810  0.5964  ...   0.4607  0.4507  0.4406

( 2 ,.,.) = 
  0.7353  0.7375  0.7404  ...   1.0000  1.0000  1.0000
  0.7367  0.7390  0.7419  ...   1.0000  1.0000  1.0000
  0.7381  0.7404  0.7434  ...   1.0000  1.0000  1.0000
           ...         


( 0 ,.,.) = 
  1.0000  1.0000  1.0000  ...   0.5572  0.5541  0.5510
  1.0000  1.0000  1.0000  ...   0.5552  0.5517  0.5481
  1.0000  1.0000  1.0000  ...   0.5531  0.5492  0.5453
           ...             ⋱             ...          
  0.0464  0.0457  0.0450  ...   0.1921  0.1430  0.0940
  0.0450  0.0443  0.0436  ...   0.1809  0.1325  0.0841
  0.0363  0.0357  0.0351  ...   0.1459  0.1068  0.0678

( 1 ,.,.) = 
  0.1230  0.1245  0.1260  ...   0.0457  0.0433  0.0349
  0.1243  0.1261  0.1279  ...   0.0441  0.0412  0.0332
  0.1257  0.1277  0.1298  ...   0.0425  0.0391  0.0315
           ...             ⋱             ...          
  0.1478  0.1420  0.1363  ...   0.2264  0.1730  0.1395
  0.1495  0.1437  0.1380  ...   0.3291  0.2745  0.2214
  0.1513  0.1457  0.1400  ...   0.4481  0.3941  0.3179

( 2 ,.,.) = 
  0.7954  0.7959  0.7957  ...   0.3588  0.3727  0.3865
  0.7898  0.7905  0.7908  ...   0.3578  0.3711  0.3844
  0.7841  0.7851  0.7858  ...   0.3568  0.3695  0.3823
           ...         

KeyboardInterrupt: 

### Function to train the model

In [42]:
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm

In [60]:
 for (i, sample) in enumerate(trainLoader):
            
            inputs = sample['image']
            labels = sample['emotion']


KeyboardInterrupt: 

In [72]:
t = tqdm(trainLoader)
        
for (i, (sample)) in enumerate(t):

# Wrap inputs, and targets into torch.autograd.Variable types.

    #################
    ##################
    #do something here (or earlier) to change # of channels in inputs from 3 to 4 
    #right now it's 64 x 224 x 224 but should be Input: (N,Cin,Hin,Win) (e.g. 64 x 1 x 224 x 224)
    ###################
    ##############
    inputs = Variable(sample['image'])
    labels = Variable(sample['emotion'])
    
    print("{}, {}".format(inputs, labels))

Variable containing:
( 0 ,.,.) = 
  0.8767  0.8720  0.8673  ...   0.6318  0.6168  0.6013
  0.8744  0.8712  0.8680  ...   0.6396  0.6245  0.6084
  0.8726  0.8702  0.8679  ...   0.6439  0.6299  0.6149
           ...             ⋱             ...          
  0.8297  0.8129  0.7961  ...   0.5754  0.5709  0.5662
  0.7821  0.7667  0.7513  ...   0.5738  0.5687  0.5633
  0.7346  0.7205  0.7065  ...   0.5723  0.5666  0.5603

( 1 ,.,.) = 
  0.3068  0.3332  0.3531  ...   0.5778  0.5734  0.5691
  0.3011  0.3285  0.3498  ...   0.6080  0.6094  0.6108
  0.2954  0.3238  0.3464  ...   0.6382  0.6454  0.6526
           ...             ⋱             ...          
  0.4041  0.4145  0.4112  ...   0.2494  0.2470  0.2447
  0.4052  0.4130  0.4066  ...   0.2534  0.2507  0.2479
  0.3974  0.4026  0.3935  ...   0.2590  0.2575  0.2560

( 2 ,.,.) = 
  0.9531  0.9343  0.9155  ...   0.8439  0.8429  0.8410
  0.9493  0.9290  0.9087  ...   0.8455  0.8443  0.8423
  0.9455  0.9237  0.9019  ...   0.8472  0.8457  0.8435
   

Variable containing:
( 0 ,.,.) = 
  0.1502  0.1566  0.1586  ...   0.0508  0.0491  0.0474
  0.1520  0.1581  0.1595  ...   0.0514  0.0475  0.0435
  0.1520  0.1580  0.1593  ...   0.0549  0.0491  0.0434
           ...             ⋱             ...          
  0.3196  0.3256  0.3205  ...   0.1519  0.1524  0.1529
  0.3224  0.3286  0.3234  ...   0.1501  0.1505  0.1509
  0.3253  0.3315  0.3263  ...   0.1483  0.1486  0.1488

( 1 ,.,.) = 
  0.0285  0.0267  0.0250  ...   0.0378  0.0400  0.0449
  0.0298  0.0277  0.0255  ...   0.0330  0.0340  0.0375
  0.0297  0.0281  0.0265  ...   0.0382  0.0386  0.0410
           ...             ⋱             ...          
  0.1645  0.1459  0.1274  ...   0.1394  0.1388  0.1447
  0.1742  0.1546  0.1349  ...   0.1526  0.1538  0.1603
  0.1840  0.1632  0.1424  ...   0.1658  0.1687  0.1758

( 2 ,.,.) = 
  0.0712  0.0937  0.1162  ...   0.3859  0.3762  0.3664
  0.0734  0.0966  0.1198  ...   0.4017  0.3896  0.3775
  0.0754  0.0992  0.1230  ...   0.4179  0.4039  0.3899
   

KeyboardInterrupt: 

In [73]:
def train_model(network, criterion, optimizer, trainLoader, valLoader, n_epochs = 10, use_gpu = False):
    if use_gpu:
        network = network.cuda()
        criterion = criterion.cuda()
        
    t_loss, t_acc, v_loss, v_acc = (np.zeros(n_epochs) for i in range(4))    
    
    # Training loop.
    for epoch in range(0, n_epochs):
        correct = 0.0
        cum_loss = 0.0
        counter = 0

        # Make a pass over the training data.
        t = tqdm(trainLoader, desc = 'Training epoch %d' % epoch)
        network.train()  # This is important to call before training!
        for (i, (sample)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(sample['image'] )
            labels = Variable(sample['emotion'])
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            # Backward pass:
            optimizer.zero_grad()
            # Loss is a variable, and calling backward on a Variable will
            # compute all the gradients that lead to that Variable taking on its
            # current value.
            loss.backward() 

            # Weight and bias updates.
            optimizer.step()

            # logging information.
            cum_loss += loss.data[0]
            max_scores, max_labels = outputs.data.max(1)
            correct += (max_labels == labels.data).sum()
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
            
        t_loss[epoch] = (cum_loss/len(t))
        t_acc[epoch] = (100*correct/counter)

        # Make a pass over the validation data.
        correct = 0.0
        cum_loss = 0.0
        counter = 0
        t = tqdm(valLoader, desc = 'Validation epoch %d' % epoch)
        network.eval()  # This is important to call before evaluating!
        for (i, (inputs, labels)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            # logging information.
            cum_loss += loss.data[0]
            max_scores, max_labels = outputs.data.max(1)
            correct += (max_labels == labels.data).sum()
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
            
        v_loss[epoch] = (cum_loss/len(t))
        v_acc[epoch] = (100*correct/counter)
        
                
    lab_utils.generate_plots(t_loss, v_loss, t_acc, v_acc, n_epochs)
            

### set learning rate, loss, optimizer, all variable stuff

In [74]:

import torch.optim as optim


#"where we set the learning rate and weight decay to 0.0005, momentum to 0.9, and dropout probability to 0.8 [25]"
learningRate = 5e-4



# Definition of our network.
#how to change the last fc layer of the model to nn.linear(4096, 7) instead of (4096, 2622)?
model.fc = nn.Linear(512, 2)

#Definition of our loss. #maybe need to change this?
criterion = nn.CrossEntropyLoss()

# Definition of optimization strategy. # maybe need to change this?
optimizer = optim.SGD(model.parameters(), lr = learningRate)

train_model(model, criterion, optimizer, trainLoader, valLoader, n_epochs = 3, use_gpu = False)

ValueError: Expected 4D tensor as input, got 3D tensor instead.